In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
import uproot
import glob

In [2]:
classes = ['QCD', 'Hbb', 'Hcc', 'Hgg', 'H4q', 'Hqql', 'Zqq', 'Wqq', 'Tbqq', 'Tbl']
n_classes = len(classes)
label_list = [f'label_{cls}' for cls in classes]
score_list = [f'score_label_{cls}' for cls in classes]

full_path = "../../results/trevmpt/pred_*.root"
#full_path = "../training/JetClass/Pythia/full/LinformerPairWise/20240919-184207_example_LinformerPairwise_ranger_lr0.001_batch512/predict_output/pred_*.root"

arrays = []
concat_arrays = {}
for file_name in glob.glob(full_path):
    with uproot.open(file_name) as f:
        print(file_name)
        arrays.append(f["Events"].arrays(label_list + score_list))
for key in label_list + score_list:
    concat_arrays[key] = np.concatenate([arrays[i][key].to_numpy() for i in range(len(arrays))])

y_prob = np.stack([concat_arrays[key] for key in score_list], axis=1)
labels = np.stack([concat_arrays[key] for key in label_list], axis=1).astype(int)

../../results/trevmpt/pred_TTBarLep.root
../../results/trevmpt/pred_HToBB.root
../../results/trevmpt/pred_ZJetsToNuNu.root
../../results/trevmpt/pred_HToWW4Q.root
../../results/trevmpt/pred_WToQQ.root
../../results/trevmpt/pred_HToGG.root
../../results/trevmpt/pred_TTBar.root
../../results/trevmpt/pred_HToCC.root
../../results/trevmpt/pred_ZToQQ.root
../../results/trevmpt/pred_HToWW2Q1L.root


In [3]:
overall_roc_auc = roc_auc_score(labels, y_prob, average='macro', multi_class='ovo')
predicted_labels = np.argmax(y_prob, axis=1) 
true_labels = np.argmax(labels, axis=1)  

accuracy = accuracy_score(true_labels, predicted_labels)

print(f'Overall ROC AUC = {overall_roc_auc:.4f}, Accuracy = {accuracy:.4f}')


scores = y_prob / (y_prob[:, :1] + y_prob) # defaults to 0.5 for QCD (not used)

rejections = []

for i in range(1, n_classes):
    if i == 5:
        percent = 0.99
    elif i == 9:
        percent = 0.995
    else:
        percent = 0.5
    
    mask = (labels[:, 0] == 1) | (labels[:, i] == 1)
    filtered_labels = labels[mask]
    filtered_scores = scores[mask]
    
    binary_labels = (filtered_labels[:, i] == 1).astype(int)
    binary_scores = filtered_scores[:, i]
    
    fpr, tpr, thresholds = roc_curve(binary_labels, binary_scores)

    idx = np.abs(tpr - percent).argmin()
    
    if fpr[idx] != 0:
        rejection = 1 / fpr[idx]
    else:
        rejection = np.inf  
    
    rejections.append(rejection)
    
    print(f'Rejection at {percent*100}% for {label_list[i]}: {rejection}')

Overall ROC AUC = 0.9877, Accuracy = 0.8606
Rejection at 50.0% for label_Hbb: 11976.047904191617
Rejection at 50.0% for label_Hcc: 4301.075268817204
Rejection at 50.0% for label_Hgg: 127.18600953895071
Rejection at 50.0% for label_H4q: 2053.388090349076
Rejection at 99.0% for label_Hqql: 5464.48087431694
Rejection at 50.0% for label_Zqq: 383.4355828220859
Rejection at 50.0% for label_Wqq: 517.8663904712585
Rejection at 50.0% for label_Tbqq: 22222.222222222223
Rejection at 99.5% for label_Tbl: 16129.032258064515
